In [1]:
import random
import time
import pygame
from numpy import sqrt, sin, cos, pi
from numpy.random import normal

SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 1000

PRINT_OUTPUT = False

# euclidean distance between two points
def distance(position1, position2):
    return ((position1[0] - position2[0]) ** 2 + (position1[1] - position2[1]) ** 2) ** 0.5

# convert a world position to a screen position
def to_screen(arena, position):
    minx, maxx = -arena.radius, arena.radius
    miny, maxy = -arena.radius, arena.radius
    x = (position[0] - minx) / (maxx - minx) * SCREEN_WIDTH
    y = (position[1] - miny) / (maxy - miny) * SCREEN_HEIGHT
    return (x, y)

# convert a world distance to a screen distance
def to_screen_distance(arena, distance):
    return (distance * SCREEN_WIDTH) // (2 * arena.radius)


##########################################################################
# Projectile types                                                       #
##########################################################################

class Projectile:
    def __init__(self, arena, attacker, target, range, range_deviation, speed, damage):
        self.arena = arena
        self.attacker = attacker
        self.target = target
        self.distance = distance(attacker.position, target.position)
        self.speed = speed
        self.range_remaining = normal(range, range_deviation)
        self.damage = damage

        # set the position and velocity of the projectile
        self.position = attacker.position
        direction = (target.position[0] - attacker.position[0], target.position[1] - attacker.position[1])
        # normalize the direction vector
        direction = (direction[0] / self.distance, direction[1] / self.distance)
        self.velocity = (direction[0] * self.speed, direction[1] * self.speed)

        # load image in depending on the size of the projectile
        size = to_screen_distance(arena, self.__class__.display_size)
        image_path = self.__class__.image_path
        self.image = pygame.image.load(image_path)
        self.image = pygame.transform.scale(self.image, (size, size))
        self.rect = self.image.get_rect()

    # return (True, _) if the projectile is used up
    # return (_, True) if the target is killed
    def update(self, delta_time):
        distance_covered = self.speed * delta_time
        self.position = (self.position[0] + self.velocity[0] * delta_time, self.position[1] + self.velocity[1] * delta_time)

        self.distance -= distance_covered
        if self.distance <= 0:
            target_killed = self.target.take_damage(self)
            return (True, target_killed)

        self.range_remaining -= distance_covered
        if self.range_remaining <= 0:
            if PRINT_OUTPUT:
                print(f'{self.attacker.name} missed {self.target.name} with {self.__class__.__name__}')
            return (True, False)

        return (False, False)

    def draw(self, screen):
        # update the position of the image
        self.rect.center = to_screen(self.arena, self.position)
        screen.blit(self.image, self.rect)


# projectile child classes
# Slash, BigSlash, Arrow, CrossbowBolt, Fireball, LightningBolt

# note that range, speed, and damage are STATIC variables
# so they are accessed by with Slash.range, Slash.speed, Slash.damage

class Slash(Projectile):
    range = 12
    range_deviation = 2
    speed = 20
    damage = 12

    display_size = 8
    image_path = 'images/slash.png'

    def __init__(self, arena, attacker, target):
        super().__init__(arena, attacker, target, Slash.range, Slash.range_deviation, Slash.speed, Slash.damage)

class BigSlash(Projectile):
    range = 25
    range_deviation = 7
    speed = 30
    damage = 24

    display_size = 16
    image_path = 'images/bigslash.png'

    def __init__(self, arena, attacker, target):
        super().__init__(arena, attacker, target, BigSlash.range, BigSlash.range_deviation, BigSlash.speed, BigSlash.damage)

class Arrow(Projectile):
    range = 150
    range_deviation = 60
    speed = 20
    damage = 6

    display_size = 4
    image_path = 'images/arrow.png'

    def __init__(self, arena, attacker, target):
        super().__init__(arena, attacker, target, Arrow.range, Arrow.range_deviation, Arrow.speed, Arrow.damage)

class CrossbowBolt(Projectile):
    range = 200
    range_deviation = 100
    speed = 30
    damage = 12

    display_size = 8
    image_path = 'images/crossbowbolt.png'

    def __init__(self, arena, attacker, target):
        super().__init__(arena, attacker, target, CrossbowBolt.range, CrossbowBolt.range_deviation, CrossbowBolt.speed, CrossbowBolt.damage)

class Fireball(Projectile):
    range = 60
    range_deviation = 20
    speed = 8
    damage = 24

    display_size = 10
    image_path = 'images/fireball.png'

    def __init__(self, arena, attacker, target):
        super().__init__(arena, attacker, target, Fireball.range, Fireball.range_deviation, Fireball.speed, Fireball.damage)

class LightningBolt(Projectile):
    range = 250
    range_deviation = 80
    speed = 80
    damage = 18

    display_size = 18
    image_path = 'images/lightning.png'

    def __init__(self, arena, attacker, target):
        super().__init__(arena, attacker, target, LightningBolt.range, LightningBolt.range_deviation, LightningBolt.speed, LightningBolt.damage)


##########################################################################
# Player types                                                           #
##########################################################################

class Player:
    def __init__(self, arena, name, position, health, attack_delay, gold):
        self.arena = arena
        self.name = name
        self.position = position
        self.health = health
        self.gold = gold
        self.kills = []

        # measured in seconds
        self.attack_delay = attack_delay
        self.time_since_last_attack = random.random() * self.attack_delay

        # load image in depending on the size of the projectile
        size = to_screen_distance(arena, self.__class__.display_size)
        image_path = self.__class__.image_path
        self.image = pygame.image.load(image_path)
        self.image = pygame.transform.scale(self.image, (size, size))
        self.rect = self.image.get_rect()

    def update(self, delta_time):
        self.time_since_last_attack += delta_time

        # if the player can't attack yet, return
        if self.time_since_last_attack < self.attack_delay:
            return None

        # otherwise, the player can attack
        self.time_since_last_attack = 0
        target, target_distance = None, None
        
        # attack the nearest enemy in the arena
        for enemy in self.arena.players:

            # don't attack yourself!
            if enemy == self:
                continue
            
            # find the distance between the player and the enemy
            dist = distance(self.position, enemy.position)

            # update the closest enemy
            if target is None or dist < target_distance:
                target, target_distance = enemy, dist

        return self.choose_attack(target)

    # this method is inherited by all child classes
    # return true if the target is killed
    def take_damage(self, projectile):
        self.health -= projectile.damage
        if self.health <= 0:
            # give the attacker the gold
            projectile.attacker.gold += self.gold
            projectile.attacker.kills.append(self.name)

            if PRINT_OUTPUT:
                print(f'{self.name} was killed by {projectile.attacker.name} using {projectile.__class__.__name__}')
                print(f'{projectile.attacker.name} now has {projectile.attacker.gold} gold')

            # return true to indicate the target is killed
            return True

        else:
            if PRINT_OUTPUT:
                print(f'{self.name} took {projectile.damage} damage from {projectile.__class__.__name__}, now has {self.health} health')
            return False

    # return a projectile object
    # this method should be overridden by child classes
    def choose_attack(self, target):
        raise NotImplementedError('Please implement this in child class.')

    def draw(self, screen):
        # update the position of the image
        self.rect.center = to_screen(self.arena, self.position)

        # draw the image
        screen.blit(self.image, self.rect)

        # display the player's name in black
        name_font = pygame.font.SysFont('Bauhaus 93', 24)
        name_text = name_font.render(self.name, True, (0, 0, 0))
        name_rect = name_text.get_rect()
        name_rect.midbottom = self.rect.midtop
        screen.blit(name_text, name_rect)

        # display the player's gold in gold
        gold_font = pygame.font.SysFont('Bauhaus 93', 20)
        gold_text = gold_font.render(f'{self.gold}', True, (240, 200, 0))
        gold_rect = gold_text.get_rect()
        gold_rect.midleft = self.rect.midright
        screen.blit(gold_text, gold_rect)

        # display the player's health bar
        remaining_health_percentage = self.health / self.__class__.health

        # health bar is 1.2x the width of the player's image
        scale = 1
        health_bar_width = self.rect.width * scale
        left_offset = (scale - 1) / 2 * self.rect.width
        remaining_health_width = health_bar_width * remaining_health_percentage

        # draw the health bar
        thickness = 8
        health_bar_rect = pygame.Rect(self.rect.left - left_offset, self.rect.bottom, health_bar_width, thickness)
        pygame.draw.rect(screen, (180, 0, 0), health_bar_rect)
        remaining_health_rect = pygame.Rect(self.rect.left - left_offset, self.rect.bottom, remaining_health_width, thickness)
        pygame.draw.rect(screen, (0, 180, 0), remaining_health_rect)


        # # display the player's health in red
        # health_font = pygame.font.SysFont('Bauhaus 93', 30)
        # health_text = health_font.render(f'{self.health}/{self.__class__.health}', True, (150, 15, 15))
        # health_rect = health_text.get_rect()
        # health_rect.midtop = self.rect.midbottom
        # screen.blit(health_text, health_rect)

# player child classes
# Skeleton, Orc, Goblin

class Skeleton(Player):
    name = 'Skeleton'
    health = 75
    attack_delay = 1.2
    gold = 10

    display_size = 10
    image_path = 'images/skeleton.png'

    def __init__(self, arena, name, position):
        name = Skeleton.name + ' ' + name
        super().__init__(arena, name, position, Skeleton.health, Skeleton.attack_delay, Skeleton.gold)

    # this is where the Skeleton AI is implemented
    def choose_attack(self, target):
        distance_to_target = distance(self.position, target.position)

        # if close enough to slash, slash
        if Slash.range >= distance_to_target:
            return Slash(self.arena, self, target)

        # otherwise, fire an arrow
        else:
            return Arrow(self.arena, self, target)

class Orc(Player):
    name = 'Orc'
    health = 100
    attack_delay = 2
    gold = 20

    display_size = 12
    image_path = 'images/orc.png'

    def __init__(self, arena, name, position):
        name = Orc.name + ' ' + name
        super().__init__(arena, name, position, Orc.health, Orc.attack_delay, Orc.gold)

    def choose_attack(self, target):
        distance_to_target = distance(self.position, target.position)

        # if close enough to slash, slash
        if Slash.range >= distance_to_target:
            return Slash(self.arena, self, target)

        # otherwise, fire a crossbow bolt
        else:
            return CrossbowBolt(self.arena, self, target)

class Goblin(Player):
    name = 'Goblin'
    health = 130
    attack_delay = 1.8
    gold = 25

    display_size = 10
    image_path = 'images/goblin.png'

    def __init__(self, arena, name, position):
        name = Goblin.name + ' ' + name
        super().__init__(arena, name, position, Goblin.health, Goblin.attack_delay, Goblin.gold)

    def choose_attack(self, target):
        distance_to_target = distance(self.position, target.position)

        # if close enough to big slash, big slash
        if BigSlash.range >= distance_to_target:
            return BigSlash(self.arena, self, target)

        # otherwise, fire an arrow
        else:
            return Arrow(self.arena, self, target)

class Boss(Player):
    def __init__(self, arena, name, position, health, healing_delay, attack_delay, gold):
        super().__init__(arena, name, position, health, attack_delay, gold)
        self.max_health = health
        self.healing_delay = healing_delay
        self.time_since_last_heal = 0

        # # scale image to 100x100 pixels
        # self.image = pygame.transform.scale(self.image, (100, 100))
        # self.rect = self.image.get_rect()

    # override the update method inherited from Player
    def update(self, delta_time):
        result = super().update(delta_time)

        # heal if enough time has passed
        self.time_since_last_heal += delta_time
        if self.time_since_last_heal >= self.healing_delay:
            self.time_since_last_heal = 0
            
            # heal by 5, up to a maximum of max_health
            new_health = min(self.health + 5, self.max_health)
            if new_health > self.health and PRINT_OUTPUT:
                print(f'{self.name} healed for {new_health - self.health} health, now has {new_health} health')
            self.health = new_health
            
        return result

class GoblinKing(Boss):
    name = 'Goblin King'
    health = 360
    healing_delay = 4
    attack_delay = 3.2
    gold = 100

    display_size = 18
    image_path = 'images/goblinking.png'

    def __init__(self, arena, name, position):
        name = GoblinKing.name + ' ' + name
        super().__init__(arena, name, position, GoblinKing.health, GoblinKing.healing_delay, GoblinKing.attack_delay, GoblinKing.gold)

    def choose_attack(self, target):
        distance_to_target = distance(self.position, target.position)

        # if close enough to big slash, big slash
        if BigSlash.range >= distance_to_target:
            return BigSlash(self.arena, self, target)
        
        # if close enough to fireball, fireball
        elif Fireball.range >= distance_to_target:
            return Fireball(self.arena, self, target)
        
        # otherwise, fire a crossbow bolt
        else:
            return CrossbowBolt(self.arena, self, target)

class Wizard(Boss):
    name = 'Wizard'
    health = 120
    healing_delay = 2.4
    attack_delay = 1.6
    gold = 120

    display_size = 20
    image_path = 'images/wizard.png'

    def __init__(self, arena, name, position):
        name = Wizard.name + ' ' + name
        super().__init__(arena, name, position, Wizard.health, Wizard.healing_delay, Wizard.attack_delay, Wizard.gold)

    def choose_attack(self, target):
        distance_to_target = distance(self.position, target.position)

        # if close enough to fireball, fireball
        if Fireball.range >= distance_to_target:
            return Fireball(self.arena, self, target)

        # otherwise, fire a lightning bolt
        else:
            return LightningBolt(self.arena, self, target)


##########################################################################
# Arena Setup                                                            #
##########################################################################

# add extra to make some letters more likely
VOWELS = 'aaaeeeiiooouuy'
C0NSONANTS = 'bbcccddfgghjklllmmnnppqrrrssssttttvwxyzz'
PLAYER_OPTIONS = [Skeleton, Orc, Goblin]
BOSS_OPTIONS = [GoblinKing, Wizard]

class Arena:
    def generate_random_name(self):
        length = random.randint(3, 6)
        start_with_vowel = random.randint(0, 1)

        name = ''
        for i in range(length):
            if (i + start_with_vowel) % 2 == 0:
                name = name + random.choice(C0NSONANTS)
            else:
                name = name + random.choice(VOWELS)
        
        return name[0].upper() + name[1:]

    def __init__(self, radius, num_players, num_bosses):
        self.radius = radius
        self.num_players = num_players
        self.players = set()
        self.dead_players = set()
        self.projectiles = []
        self.time_passed = 0

        # load in dead player image
        self.dead_player_image = pygame.image.load('images/skull.png')

        # generate players
        for i in range(num_players):
            # pick a random position in smaller 7/8 radius subcircle
            angle = random.random() * 2 * pi
            radius = sqrt(random.random()) * self.radius * 7 / 8
            position = (cos(angle) * radius, sin(angle) * radius)

            # pick a random player class
            player_class = random.choice(PLAYER_OPTIONS)

            # pick a random name
            name = self.generate_random_name()

            # create the player
            player = player_class(self, name, position)

            # add the player to the arena
            self.players.add(player)

        # generate bosses
        for i in range(num_bosses):
            # pick a random position in smaller 1/2 radius subcircle
            angle = random.random() * 2 * pi
            radius = sqrt(random.random()) * self.radius * 1 / 2
            position = (cos(angle) * radius, sin(angle) * radius)

            # pick a random boss class
            boss_class = random.choice(BOSS_OPTIONS)

            # pick a random name
            name = self.generate_random_name()

            # create the boss
            boss = boss_class(self, name, position)

            # add the boss to the arena
            self.players.add(boss)

        for player in self.players:
            # round the position to the nearest integer
            position = (round(player.position[0]), round(player.position[1]))
            # print(f'Spawned {Arena.format_player(player)}')

    def update(self, screen, delta_time):
        # update all projectiles
        i = 0
        while i < len(self.projectiles):
            projectile = self.projectiles[i]

            # check if the target is already dead
            if projectile.target.health <= 0:
                self.projectiles.pop(i)
                continue
                
            projectile_used, target_killed = projectile.update(delta_time)
            
            if target_killed:
                target = projectile.target
                self.players.remove(target)
                self.dead_players.add(target)
                if PRINT_OUTPUT:
                    self.summary()
                break

            if projectile_used:
                self.projectiles.pop(i)
            else:
                i += 1
                
        # if there is only one player left, end the game
        if len(self.players) <= 1:
            winner = None
            for player in self.players:
                winner = player

            return winner

        # update all players
        for player in self.players:
            projectile = player.update(delta_time)

            if projectile is not None:
                self.projectiles.append(projectile)

        self.time_passed += delta_time

        return False

    def draw(self, screen):
        # fill the screen
        screen.fill((20, 0, 60))

        # draw the arena as a circle
        center = to_screen(self, (0, 0))
        width = min(SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2)
        arena_color = (155, 135, 120)
        pygame.draw.circle(screen, arena_color, center, width)

        # load in dead player image
        for player in self.dead_players:
            size = to_screen_distance(self, player.display_size) * 0.4
            dead_player_image = pygame.transform.scale(self.dead_player_image, (size, size))
            screen.blit(dead_player_image, to_screen(self, player.position))
        
        # draw all players
        for player in self.players:
            player.draw(screen)

        # draw all projectiles
        for projectile in self.projectiles:
            projectile.draw(screen)
        
        pygame.display.flip()

    def summary(self):
        print(f'\n-----Summary-----')
        print(f'Time passed: {round(self.time_passed, 1)}s')
        print(f'{len(self.players)} players remain')
        for player in self.players:
            print(f'{player.name} \t{player.health} health, {player.gold} gold, killed {player.kills}')
        print('-----------------\n')

    def run(self, duration, fps=30, speed=1, print_output=False):
        PRINT_OUTPUT = print_output

        pygame.init()
        
        # set up the screen
        screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
        pygame.display.set_caption('Arena')

        # set up the clock
        clock = pygame.time.Clock()

        # print the summary
        if PRINT_OUTPUT:
            self.summary()

        # run the game for the given duration
        time_passed = 0
        delta_time = 1 / fps
        finished = False
        while time_passed < duration and not finished:
            winner = self.update(screen, delta_time)
            
            if isinstance(winner, Player):
                if PRINT_OUTPUT:
                    print(f'\n{winner.name} wins {winner.gold} gold with {winner.health} health remaining!')
                finished = True

            if winner is None:
                if PRINT_OUTPUT:
                    print(f'All players have died.')
                finished = True
            
            else: # winner is False
                pass

            time_passed += delta_time

            self.draw(screen)
            clock.tick(fps * speed)

            # handle events
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    return
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        return
        
        # # print the summary
        # self.summary()

# do this to prevent resetting arena if it's aready defined
try:
    print(arena)
except:
    arena = Arena(120, 40, 10)
arena.run(300, speed=2, print_output=False)

pygame 2.1.2 (SDL 2.0.18, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


: 